## Neural Network models - Leave One Participant Out CV to predict properties using 3 subwindows

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)
from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

import random
import datetime
import time
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
import torch.optim as optim



In [2]:
# Initialise the random state
#num = random.randint(1, 500)
num = 58
torch.manual_seed(num)
np.random.seed(num)
print(f"The generated random seed is {num}") #347

The generated random seed is 58


### Load data

In [14]:
path = "complete_dataset_"+str(15)+"subwindows_"+str(10)+"slices.csv"
df = pd.read_csv(path)
#data.iloc[12959:12965,:]



In [15]:
# Remove enjoyment as we are only considering physical properties
print(df.shape)
physical_df = df[df.property_name!='enjoyment']
physical_df.reset_index(inplace=True, drop=True)
physical_df.shape

(75588, 190)


(62989, 190)

In [16]:
existing_data = physical_df.iloc[:36000,:]
new_data = physical_df.iloc[36000:,:]

### Normalise the data

In [17]:
starting_index = 10
# Obtain a df of features
features_df = physical_df.iloc[:,starting_index:]
features_df.head()


,max_ch1_hand0,max_ch2_hand0,max_ch3_hand0,max_ch4_hand0,max_ch5_hand0,max_ch6_hand0,max_ch7_hand0,max_ch8_hand0,mean_ch1_hand0,mean_ch2_hand0,mean_ch3_hand0,mean_ch4_hand0,mean_ch5_hand0,mean_ch6_hand0,mean_ch7_hand0,mean_ch8_hand0,std_ch1_hand0,std_ch2_hand0,std_ch3_hand0,std_ch4_hand0,std_ch5_hand0,std_ch6_hand0,std_ch7_hand0,std_ch8_hand0,max_ch1_hand1,max_ch2_hand1,max_ch3_hand1,max_ch4_hand1,max_ch5_hand1,max_ch6_hand1,max_ch7_hand1,max_ch8_hand1,mean_ch1_hand1,mean_ch2_hand1,mean_ch3_hand1,mean_ch4_hand1,mean_ch5_hand1,mean_ch6_hand1,mean_ch7_hand1,mean_ch8_hand1,std_ch1_hand1,std_ch2_hand1,std_ch3_hand1,std_ch4_hand1,std_ch5_hand1,std_ch6_hand1,std_ch7_hand1,std_ch8_hand1,max_Ax_hand0,max_Ay_hand0,max_Az_hand0,max_Vx_hand0,max_Vy_hand0,max_Vz_hand0,max_Jx_hand0,max_Jy_hand0,max_Jz_hand0,mean_Ax_hand0,mean_Ay_hand0,mean_Az_hand0,mean_Vx_hand0,mean_Vy_hand0,mean_Vz_hand0,mean_Jx_hand0,mean_Jy_hand0,mean_Jz_hand0,std_Ax_hand0,std_Ay_hand0,std_Az_hand0,std_Vx_hand0,std_Vy_hand0,std_Vz_hand0,std_Jx_hand0,std_Jy_hand0,std_Jz_hand0,max_Ax_hand1,max_Ay_hand1,max_Az_hand1,max_Vx_hand1,max_Vy_hand1,max_Vz_hand1,max_Jx_hand1,max_Jy_hand1,max_Jz_hand1,mean_Ax_hand1,mean_Ay_hand1,mean_Az_hand1,mean_Vx_hand1,mean_Vy_hand1,mean_Vz_hand1,mean_Jx_hand1,mean_Jy_hand1,mean_Jz_hand1,std_Ax_hand1,std_Ay_hand1,std_Az_hand1,std_Vx_hand1,std_Vy_hand1,std_Vz_hand1,std_Jx_hand1,std_Jy_hand1,std_Jz_hand1,max_w_hand0,max_x_hand0,max_y_hand0,max_z_hand0,max_AVx_hand0,max_AVy_hand0,max_AVz_hand0,max_AAx_hand0,max_AAy_hand0,max_AAz_hand0,max_AJx_hand0,max_AJy_hand0,max_AJz_hand0,mean_w_hand0,mean_x_hand0,mean_y_hand0,mean_z_hand0,mean_AVx_hand0,mean_AVy_hand0,mean_AVz_hand0,mean_AAx_hand0,mean_AAy_hand0,mean_AAz_hand0,mean_AJx_hand0,mean_AJy_hand0,mean_AJz_hand0,std_w_hand0,std_x_hand0,std_y_hand0,std_z_hand0,std_AVx_hand0,std_AVy_hand0,std_AVz_hand0,std_AAx_hand0,std_AAy_hand0,std_AAz_hand0,std_AJx_hand0,std_AJy_hand0,std_AJz_hand0,max_w_hand1,max_x_hand1,max_y_hand1,max_z_hand1,max_AVx_hand1,max_AVy_hand1,max_AVz_hand1,max_AAx_hand1,max_AAy_hand1,max_AAz_hand1,max_AJx_hand1,max_AJy_hand1,max_AJz_hand1,mean_w_hand1,mean_x_hand1,mean_y_hand1,mean_z_hand1,mean_AVx_hand1,mean_AVy_hand1,mean_AVz_hand1,mean_AAx_hand1,mean_AAy_hand1,mean_AAz_hand1,mean_AJx_hand1,mean_AJy_hand1,mean_AJz_hand1,std_w_hand1,std_x_hand1,std_y_hand1,std_z_hand1,std_AVx_hand1,std_AVy_hand1,std_AVz_hand1,std_AAx_hand1,std_AAy_hand1,std_AAz_hand1,std_AJx_hand1,std_AJy_hand1,std_AJz_hand1
0,0.5,0.615385,0.670391,0.495868,0.759036,0.652406,0.75625,0.666667,0.497805,0.613462,0.665764,0.492188,0.718656,0.636197,0.748926,0.660851,0.002324,0.002503,0.003077,0.002455,0.006501,0.004039,0.003061,0.002083,0.508197,0.908451,0.751445,0.730159,0.493878,0.995918,0.648515,0.576037,0.496670,0.846831,0.691595,0.636244,0.491412,0.608674,0.601382,0.558852,0.004198,0.014491,0.015432,0.026432,0.002186,0.218816,0.022186,0.006429,-0.857056,-0.084885,0.479568,0.000208,0.000215,0.000349,0.718061,0.488281,1.206342,-0.863110,-0.091721,0.468826,9.082031e-06,-0.000028,-0.000006,0.036854,0.039343,0.092439,0.006753,0.007619,0.010341,0.000193,0.000245,0.000281,0.488563,0.441082,0.750762,0.986160,0.013489,-0.211105,0.001218,0.000510,0.000937,1.448006,0.708008,2.343750,0.964774,-0.007117,-0.249875,1.117188e-04,0.000070,0.000027,-0.104780,0.096063,0.335256,0.022364,0.013963,0.028878,0.000738,0.000331,0.000943,1.260192,0.571902,1.565671,0.179352,0.484946,0.160615,-0.840857,0.009831,0.012959,0.117029,1.270143,2.238883,19.812834,154.113175,104.818227,1338.775861,0.178765,0.484513,0.160469,-0.841154,-0.016003,-0.027618,-0.081836,0.129868,0.552890,-1.016133,16.779031,-45.915024,210.907738,0.000451,0.000472,0.000210,0.000335,0.016423,0.031903,0.200440,1.160057,1.546528,13.337295,97.220032,155.178181,1114.885458,0.600343,-0.175433,-0.769104,-0.129347,0.140072,0.440644,0.126734,5.745156,20.612016,5.331893,212.817696,779.034203,579.105295,0.598260,-0.176827,-0.770695,-0.129768,0.053843,0.125821,-0.043329,0.487977,0.515185,

In [18]:
normalised_features_df = features_df.copy()
print(normalised_features_df.shape)

(62989, 180)


In [19]:
# create scaler
scaler = MinMaxScaler(feature_range=(-1,1)) # As this is the range of the activation function - tanh

# fit scaler and apply transform
normalised_features_df[normalised_features_df.columns] = scaler.fit_transform(features_df[features_df.columns])
normalised_features_df.shape

(62989, 180)

In [20]:
df_info = physical_df.iloc[:, :starting_index]
df_info.head()

,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,slice_num
0,7,14,0,smoothness,1,3,medium,2,1,1
1,7,14,0,smoothness,1,3,medium,2,1,2
2,7,14,0,smoothness,1,3,medium,2,1,3
3,7,14,0,smoothness,1,3,medium,2,1,4
4,7,14,0,smoothness,1,3,medium,2,1,5


In [21]:
normalised_df = pd.concat([df_info,normalised_features_df], axis=1)
normalised_df.shape

(62989, 190)

In [22]:
normalised_lili = normalised_df.iloc[:10800,:]
normalised_dat_2022 = normalised_df.iloc[10800:,:]


In [23]:
existing_data = physical_df.iloc[:36000,:]
new_data = physical_df.iloc[36000:,:]

existing_data.head(2)


,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,slice_num,max_ch1_hand0,max_ch2_hand0,max_ch3_hand0,max_ch4_hand0,max_ch5_hand0,max_ch6_hand0,max_ch7_hand0,max_ch8_hand0,mean_ch1_hand0,mean_ch2_hand0,mean_ch3_hand0,mean_ch4_hand0,mean_ch5_hand0,mean_ch6_hand0,mean_ch7_hand0,mean_ch8_hand0,std_ch1_hand0,std_ch2_hand0,std_ch3_hand0,std_ch4_hand0,std_ch5_hand0,std_ch6_hand0,std_ch7_hand0,std_ch8_hand0,max_ch1_hand1,max_ch2_hand1,max_ch3_hand1,max_ch4_hand1,max_ch5_hand1,max_ch6_hand1,max_ch7_hand1,max_ch8_hand1,mean_ch1_hand1,mean_ch2_hand1,mean_ch3_hand1,mean_ch4_hand1,mean_ch5_hand1,mean_ch6_hand1,mean_ch7_hand1,mean_ch8_hand1,std_ch1_hand1,std_ch2_hand1,std_ch3_hand1,std_ch4_hand1,std_ch5_hand1,std_ch6_hand1,std_ch7_hand1,std_ch8_hand1,max_Ax_hand0,max_Ay_hand0,max_Az_hand0,max_Vx_hand0,max_Vy_hand0,max_Vz_hand0,max_Jx_hand0,max_Jy_hand0,max_Jz_hand0,mean_Ax_hand0,mean_Ay_hand0,mean_Az_hand0,mean_Vx_hand0,mean_Vy_hand0,mean_Vz_hand0,mean_Jx_hand0,mean_Jy_hand0,mean_Jz_hand0,std_Ax_hand0,std_Ay_hand0,std_Az_hand0,std_Vx_hand0,std_Vy_hand0,std_Vz_hand0,std_Jx_hand0,std_Jy_hand0,std_Jz_hand0,max_Ax_hand1,max_Ay_hand1,max_Az_hand1,max_Vx_hand1,max_Vy_hand1,max_Vz_hand1,max_Jx_hand1,max_Jy_hand1,max_Jz_hand1,mean_Ax_hand1,mean_Ay_hand1,mean_Az_hand1,mean_Vx_hand1,mean_Vy_hand1,mean_Vz_hand1,mean_Jx_hand1,mean_Jy_hand1,mean_Jz_hand1,std_Ax_hand1,std_Ay_hand1,std_Az_hand1,std_Vx_hand1,std_Vy_hand1,std_Vz_hand1,std_Jx_hand1,std_Jy_hand1,std_Jz_hand1,max_w_hand0,max_x_hand0,max_y_hand0,max_z_hand0,max_AVx_hand0,max_AVy_hand0,max_AVz_hand0,max_AAx_hand0,max_AAy_hand0,max_AAz_hand0,max_AJx_hand0,max_AJy_hand0,max_AJz_hand0,mean_w_hand0,mean_x_hand0,mean_y_hand0,mean_z_hand0,mean_AVx_hand0,mean_AVy_hand0,mean_AVz_hand0,mean_AAx_hand0,mean_AAy_hand0,mean_AAz_hand0,mean_AJx_hand0,mean_AJy_hand0,mean_AJz_hand0,std_w_hand0,std_x_hand0,std_y_hand0,std_z_hand0,std_AVx_hand0,std_AVy_hand0,std_AVz_hand0,std_AAx_hand0,std_AAy_hand0,std_AAz_hand0,std_AJx_hand0,std_AJy_hand0,std_AJz_hand0,max_w_hand1,max_x_hand1,max_y_hand1,max_z_hand1,max_AVx_hand1,max_AVy_hand1,max_AVz_hand1,max_AAx_hand1,max_AAy_hand1,max_AAz_hand1,max_AJx_hand1,max_AJy_hand1,max_AJz_hand1,mean_w_hand1,mean_x_hand1,mean_y_hand1,mean_z_hand1,mean_AVx_hand1,mean_AVy_hand1,mean_AVz_hand1,mean_AAx_hand1,mean_AAy_hand1,mean_AAz_hand1,mean_AJx_hand1,mean_AJy_hand1,mean_AJz_hand1,std_w_hand1,std_x_hand1,std_y_hand1,std_z_hand1,std_AVx_hand1,std_AVy_hand1,std_AVz_hand1,std_AAx_hand1,std_AAy_hand1,std_AAz_hand1,std_AJx_hand1,std_AJy_hand1,std_AJz_hand1
0,7,14,0,smoothness,1,3,medium,2,1,1,0.5,0.615385,0.670391,0.495868,0.759036,0.652406,0.75625,0.666667,0.497805,0.613462,0.665764,0.492188,0.718656,0.636197,0.748926,0.660851,0.002324,0.002503,0.003077,0.002455,0.006501,0.004039,0.003061,0.002083,0.508197,0.908451,0.751445,0.730159,0.493878,0.995918,0.648515,0.576037,0.496670,0.846831,0.691595,0.636244,0.491412,0.608674,0.601382,0.558852,0.004198,0.014491,0.015432,0.026432,0.002186,0.218816,0.022186,0.006429,-0.857056,-0.084885,0.479568,0.000208,0.000215,0.000349,0.718061,0.488281,1.206342,-0.863110,-0.091721,0.468826,9.082031e-06,-0.000028,-0.000006,0.036854,0.039343,0.092439,0.006753,0.007619,0.010341,0.000193,0.000245,0.000281,0.488563,0.441082,0.750762,0.98616,0.013489,-0.211105,0.001218,0.000510,0.000937,1.448006,0.708008,2.343750,0.964774,-0.007117,-0.249875,0.000112,0.000070,0.000027,-0.104780,0.096063,0.335256,0.022364,0.013963,0.028878,0.000738,0.000331,0.000943,1.260192,0.571902,1.565671,0.179352,0.484946,0.160615,-0.840857,0.009831,0.012959,0.117029,1.270143,2.238883,19.812834,154.113175,104.818227,1338.775861,0.178765,0.484513,0.160469,-0.841154,-0.016003,-0.027618,-0.081836,0.129868,0.552890,-1.016133,16.779031,-45.915024,210.907738,0.000451,0.000472,0.000210,0.000335,0.016423,0.031903,0.200440,1.160057,1.546528,13.337295,97.220032,155.178181,1114.885458,0.600343,-0.175433,-0.769104,-0.129347,0.140072,0.440

In [24]:
starting_index = 10

# Create a df of features
existing_features_df = existing_data.iloc[:,starting_index:]
new_features_df = new_data.iloc[:,starting_index:]

# Create a df with the first 10 columns
existing_info = existing_data.iloc[:, :starting_index]
new_info = new_data.iloc[:, :starting_index]

normalised_existing_features = existing_features_df.copy()
normalised_new_features = new_features_df.copy()
print(normalised_existing_features.shape)
print(normalised_new_features.shape)

# create scaler
scaler = MinMaxScaler(feature_range=(-1,1)) # As this is the range of the activation function - tanh

# fit scaler and apply transform
normalised_existing_features[normalised_existing_features.columns] = scaler.fit_transform(existing_features_df[existing_features_df.columns])
normalised_new_features[normalised_new_features.columns] = scaler.fit_transform(new_features_df[new_features_df.columns])

print(normalised_existing_features.shape)
print(normalised_new_features.shape)

normalised_existing_df = pd.concat([existing_info, normalised_existing_features], axis=1)
normalised_new_df = pd.concat([new_info, normalised_new_features], axis=1)




(36000, 180)
(26989, 180)
(36000, 180)
(26989, 180)


### Create X and y data

In [25]:
def create_X_2d(df, features_starting_idx):
    
    X_2d = df.iloc[:,features_starting_idx:].values
    
    X_tensor_2d = torch.Tensor(X_2d)    
    return X_tensor_2d



In [26]:
def create_y_train_for_2d_X(df, predicting_feature = 'property_id', output_as_tensor='Yes'):
    # CreatE an instance of a one-hot-encoder
    encoder = OneHotEncoder(handle_unknown='ignore')

    # Perform one-hot encoding on the specified column 
    encoder_df = pd.DataFrame(encoder.fit_transform(df[[predicting_feature]]).toarray())
    
    # Convert to a numpy array
    y_train = encoder_df.to_numpy()
    
    if output_as_tensor == 'Yes':
        # Convert to a tensor
        y_train = torch.Tensor(y_train)

    return y_train



In [27]:
def create_y_test_for_2d_X(df, predicting_feature = 'property_id'):   
    y_test = df[predicting_feature].values
    #if predicting_feature == 'property_id':
       # y_test = y_test - 3
    if predicting_feature == 'rating_level_num':
        y_test = y_test - 1
    
    y_test_tensor = torch.Tensor(y_test)    
    y_test_tensor = y_test_tensor.type(torch.LongTensor)
    
    return y_test_tensor


In [28]:
def create_X_3d(df, features_starting_idx):
    dim1 = df.new_interaction_id.nunique()
    print(dim1)
    dim2 = df.slice_num.nunique()
    dim3 = df.iloc[:,features_starting_idx:].shape[1]
        
    X = np.zeros((dim1, dim2, dim3)) 

    itr_id_lst = df.new_interaction_id.unique().tolist()
    print(itr_id_lst[0], itr_id_lst[-1])

    for itr_id in itr_id_lst: #range(len(itr_id_lst)):
        #itr_id = itr_id_lst[i]
        itr_id_df = df[df.new_interaction_id==itr_id]  
        
        for j in range(itr_id_df.shape[0]):
            vals_arr = itr_id_df.iloc[j,features_starting_idx:].values
            if itr_id-1 == dim1:
                print(itr_id)
            X[itr_id-1,j] = vals_arr
    
    X_tensor = torch.Tensor(X)    
    return X_tensor


In [29]:
def create_y_train_for_3d_X(df, predicting_feature = 'property_id'):
    # Create a dataset with only the required columns
    df2 = df[['new_interaction_id', 'property_id', 'rating_level_num']]

    # Remove duplicates
    df2.drop_duplicates(keep='first', inplace=True)

    # Reset the indexes
    df2.reset_index(drop=True, inplace=True) 
    
    ## Create y train
    # CreatE an instance of a one-hot-encoder
    encoder = OneHotEncoder(handle_unknown='ignore')

    # Perform one-hot encoding on the specified column 
    encoder_df = pd.DataFrame(encoder.fit_transform(df2[[predicting_feature]]).toarray())
    
    # Convert to a numpy array
    y_train = encoder_df.to_numpy()
    
    # Convert to a tensor
    y_train = torch.Tensor(y_train)
  
    return y_train



In [30]:
def create_y_test_for_3d_X(df, predicting_feature = 'property_id'):
    # Create a dataset with only the required columns
    df2 = df[['new_interaction_id', 'property_id', 'rating_level_num']]

    # Remove duplicates
    df2.drop_duplicates(keep='first', inplace=True)

    # Reset the indexes
    df2.reset_index(drop=True, inplace=True) 
    
    y_test = df2[predicting_feature].values
    #if predicting_feature == 'property_id':
       # y_test = y_test - 3
    if predicting_feature == 'rating_level_num':
        y_test = y_test - 1
    
    y_test = torch.Tensor(y_test)    
    y_test = y_test.type(torch.LongTensor)
    
    return y_test



## Model 1 - Linear model using all 180 features

### Create the model

In [31]:
class Linear_all_features_properties(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(180, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 5)        
           
    def forward(self, x): #, x2
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

class Linear_all_features_ratings(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(185, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 3)        

    def forward(self, x): #, x2
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x
 

In [32]:
class LSTM_all_features_properties(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.LSTM(90, 40, 1, batch_first=True)
        self.fc1 = nn.Linear(10 * 40 * 2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 5)
          
    def forward(self, x1, x2): #, x2
        #print(x1.shape)
        x1, (hn, cn) = self.rnn(x1) #, (self.h0, self.c0)
        x1 = F.tanh(x1)
        #print(x2.shape)
        x2, (hm, cm) = self.rnn(x2) # (self.h0, self.c0)
        x2 = F.tanh(x2)
        #print(x1.shape,x2.shape)
        x = torch.cat((x1, x2), 2)
        #print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x
    

In [33]:
def find_best_model_for_2d_X(train_dataloader, learning_rate, num_epochs, model):

    # Model
    train_model = model

    # Loss and Optimiser
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.7)

    best_train_loss = np.inf
    best_model = None
    #best_model_epoch_num = np.inf
    train_loss_lst = []
    #val_loss_lst = []
    #avg_loss_lst = []

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        #Set the model in training mode
        train_model.train()

        # Initialise the total training and validation loss
        epoch_train_loss = 0
        epoch_val_loss = 0
        avg_loss = 0

        for i, train_data in enumerate(train_dataloader, 0):

            # get the inputs; data is a list of [input1, input2, label]
            train_input1, train_labels = train_data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_preds = train_model(train_input1) #, train_input2
            train_loss = criterion(train_preds, train_labels)
            train_loss.backward()
            optimizer.step()

            # Update training loss
            epoch_train_loss += train_loss.item()
      
        avg_training_loss = epoch_train_loss / len(train_dataloader) 
        
        train_loss_lst.append(avg_training_loss)
         
        if epoch % 10 == 0:
            print(f'epoch {epoch+1}: train loss = {round(avg_training_loss,2)}')

        if avg_training_loss < best_train_loss:
            best_train_loss = avg_training_loss
            best_model = train_model.state_dict()

    return best_train_loss, best_model, train_loss_lst   #, avg_loss_lst, 


In [34]:
def find_best_model_for_3d_X(train_dataloader, learning_rate, num_epochs, model):

    # Model
    train_model = model

    # Loss and Optimiser
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.7)

    best_avg_loss = np.inf
    best_model = None
    #best_model_epoch_num = np.inf
    train_loss_lst = []
   # val_loss_lst = []
   # avg_loss_lst = []

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        #Set the model in training mode
        train_model.train()

        # Initialise the total training and validation loss
        epoch_train_loss = 0
        epoch_val_loss = 0
        avg_loss = 0

        #running_loss = 0.0
        for i, train_data in enumerate(train_dataloader, 0):
            #print(len(train_data))

            # get the inputs; data is a list of [input1, input2, label]
            train_input1, train_input2, train_labels = train_data #train_input2, 

            #train_labels = train_labels.type(torch.LongTensor)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_preds = train_model(train_input1, train_input2)  
            #print(train_labels)#

            train_loss = criterion(train_preds, train_labels)
            train_loss.backward()
            optimizer.step()

            # Update training loss
            epoch_train_loss += train_loss.item()

     
        avg_training_loss = epoch_train_loss / len(train_dataloader) #count_train
        
        train_loss_lst.append(avg_training_loss)
         
        #print(f'epoch {epoch+1}: train loss = {round(avg_training_loss,3)}, val loss = {round(avg_validation_loss,3)}, average loss = {round(avg_loss,3)}')
        if epoch % 10 == 0:
            print(f'epoch {epoch+1}: train loss = {round(avg_training_loss,2)}')
        
        if avg_training_loss < best_train_loss:
            best_train_loss = avg_training_loss
            best_model = train_model.state_dict()

    return best_train_loss, best_model, train_loss_lst 

    #return best_avg_loss, best_model, train_loss_lst, val_loss_lst, avg_loss_lst   #, avg_loss_lst, 


In [35]:
# Original
def Linear_LOP0CV(lili_data, dat_2022, model, num_folds=5, predicting_feature='property_id', learning_rate=0.01, num_epochs=10, random_state=num): #, num_inner_folds=5
    # Set fixed random number seed
    torch.manual_seed(num)
    
    total_conf_mat = 0
    micro_f1_lst = []
    acc_lst = []
    if predicting_feature == 'property_id':
        macro_f1_lst = []
    elif predicting_feature == 'rating_level_num':
        weighted_f1_lst = []  
        
    # Data preparation
    X_train = create_X_2d(lili_data, 10)       
    y_train = create_y_train_for_2d_X(lili_data, predicting_feature = predicting_feature, output_as_tensor='Yes')
    X_test = create_X_2d(dat_2022, 10) 
    y_test = create_y_test_for_2d_X(dat_2022, predicting_feature = predicting_feature)
        
    if predicting_feature == 'rating_level_num':
        X_train_add = create_y_train_for_2d_X(lili_data, predicting_feature = 'property_id', output_as_tensor='Yes')
        X_train = torch.cat((X_train, X_train_add), 1)
        X_test_add = create_y_train_for_2d_X(dat_2022, predicting_feature = 'property_id', output_as_tensor='Yes')  
        X_test = torch.cat((X_test, X_test_add), 1)
    
    train_dataset = TensorDataset(X_train, y_train)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=y_train.shape[0]) # num_workers=2,
    
    test_dataset = TensorDataset(X_test, y_test)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=y_test.shape[0]) # num_workers=2,       

    avg_training_loss, best_model, training_loss_lst = find_best_model_for_2d_X(train_dataloader, learning_rate, num_epochs, model)

    # save trained model 
    name = 'best_model.pt'
    torch.save(best_model, name)
    print(f'The model has been saved')

    test_model = model
    test_model.load_state_dict(torch.load(name))

    dataiter = iter(test_dataloader) 
    test_input1, test_labels = dataiter.next() 

    test_preds = test_model(test_input1) 

    test_preds_np = test_preds.detach().numpy()
    test_predicted_np = np.argmax(test_preds_np, axis = 1)

    test_labels_np = test_labels.numpy()       

    if predicting_feature == 'property_id':
        conf_mat = confusion_matrix(test_labels_np, test_predicted_np, labels=[0, 1, 2, 3, 4])
        macro_f1_score = f1_score(test_labels_np, test_predicted_np, average='macro') 
        macro_f1_lst.append(macro_f1_score) 
    elif predicting_feature == 'rating_level_num':
        conf_mat = confusion_matrix(test_labels_np, test_predicted_np, labels=[0,1,2])
        weighted_f1_score = f1_score(test_labels_np, test_predicted_np, average='weighted') 
        weighted_f1_lst.append(weighted_f1_score)

    total_conf_mat += conf_mat
    micro_f1_score = f1_score(test_labels_np, test_predicted_np, average='micro')  
    micro_f1_lst.append(micro_f1_score)
    acc = accuracy_score(test_labels_np, test_predicted_np)
    acc_lst.append(acc)


    #print(f"Leaving participant {participant} out")
    print("(1) Confusion matrix:\n", conf_mat)
    print(f"(2) micro F1 score = {round(micro_f1_score,2)}") 
    if predicting_feature == 'property_id':
        print(f"(3) Macro F1 score = {round(macro_f1_score,2)}")
    elif predicting_feature == 'rating_level_num':
        print(f"(3) Weighted F1 score = {round(weighted_f1_score,2)}")            
    print(f"(4) Percentage Classification accuracy = {round(acc*100,2)}%")

    print('--------------------------------')
        

In [36]:
# Original
def LSTM_LOP0CV(lili_0, lili_1, new_0, new_1, model, num_folds=5, predicting_feature='property_id', learning_rate=0.01, num_epochs=10, random_state=num): #, num_inner_folds=5
    # Set fixed random number seed
    torch.manual_seed(num)
        
    total_conf_mat = 0
    micro_f1_lst = []
    acc_lst = []
    if predicting_feature == 'property_id':
        macro_f1_lst = []
    elif predicting_feature == 'rating_level_num':
        weighted_f1_lst = [] 
        
        data0_add = create_y_train_for_2d_X(data_0, predicting_feature = 'property_id', output_as_tensor='No')
        data0_add_pd = pd.DataFrame(data0_add, columns = ['smoothness','thickness','warmth', 'flexibility', 'softness'])
        data_0 = pd.concat([data_0.reset_index(drop=True), data0_add_pd.reset_index(drop=True)], axis=1)
        data1_add = create_y_train_for_2d_X(data_1, predicting_feature = 'property_id', output_as_tensor='No')
        data1_add_pd = pd.DataFrame(data1_add, columns = ['smoothness','thickness','warmth', 'flexibility', 'softness'])
        data_1 = pd.concat([data_1.reset_index(drop=True), data1_add_pd.reset_index(drop=True)], axis=1)
    
    #for participant in sorted(data_0.participant_id.unique()): # # #:lst: #
       # print(f'LEAVING PARTICIPANT {participant} OUT:')
        
        # Split the data into training and testing
        #training_data_0 = data_0[data_0.participant_id != participant] 
        #training_data_1 = data_1[data_1.participant_id != participant] 
        #testing_data_0 = data_0[data_0.participant_id == participant] 
        #testing_data_1 = data_1[data_1.participant_id == participant] 

    # Data preparation
    X_train_0 = create_X_3d(lili_0, 10)
    X_train_1 = create_X_3d(lili_1, 10) 
    X_test_0 = create_X_3d(new_0, 10) 
    X_test_1 = create_X_3d(new_1, 10)           
    y_train = create_y_train_for_3d_X(lili_0, predicting_feature = predicting_feature)
    y_test = create_y_test_for_3d_X(new_0, predicting_feature = predicting_feature)        


        #print(X_train_0.shape)
        #print(X_train_1.shape)
        #print(y_train.shape)
        #print(X_test_0.shape)
        #print(X_test_1.shape)
        #print(y_test.shape)
 
    # Create the datasets and dataloaders
    train_dataset = TensorDataset(X_train_0, X_train_1, y_train) 
    train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=y_train.shape[0]) # num_workers=2,

    test_dataset = TensorDataset(X_test_0, X_test_1, y_test)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=y_test.shape[0]) # num_workers=2,       
            
    train_loss, best_model, train_loss_lst = find_best_model_for_3d_X(train_dataloader, learning_rate, num_epochs, model)
                            
    # save trained model 
    name = 'best_model.pt'
    torch.save(best_model, name)
    print(f'The model has been saved')
      
        
    test_model = model
    test_model.load_state_dict(torch.load(name))

    dataiter = iter(test_dataloader) 
    test_input1, test_input2, test_labels = dataiter.next() 

    test_preds = test_model(test_input1, test_input2) 

    test_preds_np = test_preds.detach().numpy()
    test_predicted_np = np.argmax(test_preds_np, axis = 1)

    test_labels_np = test_labels.numpy()    
    
    if predicting_feature == 'property_id':
        conf_mat = confusion_matrix(test_labels_np, test_predicted_np, labels=[0, 1, 2, 3, 4])
        macro_f1_score = f1_score(test_labels_np, test_predicted_np, average='macro') 
        #macro_f1_lst.append(macro_f1_score) 
    elif predicting_feature == 'rating_level_num':
        conf_mat = confusion_matrix(test_labels_np, test_predicted_np, labels=[0,1,2])
        weighted_f1_score = f1_score(test_labels_np, test_predicted_np, average='weighted') 
        weighted_f1_lst.append(weighted_f1_score)

    #total_conf_mat += conf_mat
    micro_f1_score = f1_score(test_labels_np, test_predicted_np, average='micro')  
    #micro_f1_lst.append(micro_f1_score)
    acc = accuracy_score(test_labels_np, test_predicted_np)
    #acc_lst.append(acc)


    #print(f"Leaving participant {participant} out")
    print("(1) Confusion matrix:\n", conf_mat)
    print(f"(2) micro F1 score = {round(micro_f1_score,2)}") 
    if predicting_feature == 'property_id':
        print(f"(3) Macro F1 score = {round(macro_f1_score,2)}")
    elif predicting_feature == 'rating_level_num':
        print(f"(3) Weighted F1 score = {round(weighted_f1_score,2)}")            
    print(f"(4) Percentage Classification accuracy = {round(acc*100,2)}%")

    print('--------------------------------')
        
   

### Run the FC for properties

In [49]:
t1 = time.time()
Linear_LOP0CV(normalised_lili, normalised_dat_2022, model=Linear_all_features_properties(), num_folds=5, predicting_feature='property_id', learning_rate=0.05, num_epochs=5000, random_state=num)
t2 = time.time()

epoch 1: train loss = 1.61
epoch 11: train loss = 1.61
epoch 21: train loss = 1.61
epoch 31: train loss = 1.61
epoch 41: train loss = 1.61
epoch 51: train loss = 1.61
epoch 61: train loss = 1.61
epoch 71: train loss = 1.61
epoch 81: train loss = 1.61
epoch 91: train loss = 1.61
epoch 101: train loss = 1.61
epoch 111: train loss = 1.61
epoch 121: train loss = 1.61
epoch 131: train loss = 1.61
epoch 141: train loss = 1.61
epoch 151: train loss = 1.61
epoch 161: train loss = 1.61
epoch 171: train loss = 1.61
epoch 181: train loss = 1.61
epoch 191: train loss = 1.61
epoch 201: train loss = 1.61
epoch 211: train loss = 1.61
epoch 221: train loss = 1.61
epoch 231: train loss = 1.61
epoch 241: train loss = 1.61
epoch 251: train loss = 1.61
epoch 261: train loss = 1.61
epoch 271: train loss = 1.61
epoch 281: train loss = 1.61
epoch 291: train loss = 1.61
epoch 301: train loss = 1.61
epoch 311: train loss = 1.61
epoch 321: train loss = 1.61
epoch 331: train loss = 1.61
epoch 341: train loss = 1

In [51]:
t2-t1

384.22882103919983